In [5]:
import numpy as np
from astropy import units as u
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import scopesim as sim
from scopesim_templates.micado.spectral_calibrations import line_list

# Comment out if you used scopesim.download_packages(), 
# or change to the path of your local cloned version of the github IRDB directory
sim.rc.__config__["!SIM.file.local_packages_path"] = "F:/Work/irdb"

### Make the line list input Source object

In [6]:
# unit_flux means a relative intensity of 1 = 3000 PHOTLAM [ph s-1 cm-2 AA-1]
# dwave = 1e-3 [nm] as discussed on slack --> change this to 1e-2 to speed up scopesim (maybe)
src_lines = line_list(unit_flux=3000, dwave=1e-3)

### Make the spectroscopic mode optical model

In [8]:
cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", "SPEC_3000x50"])
cmds["!OBS.dit"] = 3600
cmds["!OBS.filter_name_fw1"] = "Spec_HK"     # Spec_IJ, Spec_HK
cmds["!OBS.filter_name_fw2"] = "open"


micado = sim.OpticalTrain(cmds)

# Set to False to test if everything is working, then flip to True
USE_FULL_DETECTOR = False
micado["full_detector_array"].include = USE_FULL_DETECTOR
micado["detector_window"].include = not USE_FULL_DETECTOR

In [ ]:
# This makes the perfect noiseless focal plane image
# This may take a while: ~order of mins
micado.observe(src_lines)

### Plot the intermediate noiseless focal plane image

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(micado.image_planes[0].data, norm=LogNorm())

### This adds detector and shot noise

In [ ]:
hdu = micado.readout()[0]

In [ ]:
# The intermediate and detector images are here in memory
# If you want to write them to disk, do this:
micado.image_planes[0].hdu.writeto("img_plane_intermediate.fits")
hdu.writeto("detector_final.fits")

In [ ]:
src_lines = line_list()

waves = np.arange(0.8, 2.5, 1e-7) * u.um
flux = src_lines.spectra[0](waves)

plt.plot(waves, flux)

### Display information about the elements in the MICADO optical train

In [9]:
# List all the effects included in MICADO
micado.effects

element,name,class,included
str11,str27,str24,bool
armazones,skycalc_atmosphere,SkycalcTERCurve,True
ELT,telescope_reflection,SurfaceList,True
MICADO,micado_static_surfaces,SurfaceList,True
MICADO,micado_ncpas_psf,NonCommonPathAberration,True
MICADO,filter_wheel_1 : [Spec_HK],FilterWheel,True
MICADO,filter_wheel_2 : [open],FilterWheel,True
MICADO,pupil_wheel : [open],FilterWheel,True
MICADO_DET,full_detector_array,DetectorList,False
MICADO_DET,detector_window,DetectorWindow,True


In [17]:
# Detailed info about each effect can be found in the .meta dictionaries
micado["filter_wheel_1"].meta

{'filename': None,
 'description': '',
 'history': [],
 'name': 'filter_wheel_2',
 'temperature': -190,
 'filter_file_format': 'filters/TC_filter_{}.dat',
 'element_name': 'MICADO',
 'filter_names': ['open',
  'xY2',
  'xK1',
  'xK2',
  'block',
  'K-mid',
  'K-short',
  'Ks',
  'H',
  'H-long',
  'H-short',
  'xH1',
  'xH2',
  'xJ1',
  'xJ2',
  'xY1'],
 'filename_format': '!INST.filter_file_format',
 'current_filter': '!OBS.filter_name_fw2',
 'minimum_throughput': 0.000101,
 'outer': 0.2,
 'outer_unit': 'm',
 'z_order': [124, 224, 524],
 'include': True,
 'path': '',
 'report_plot_include': True,
 'report_table_include': True,
 'report_table_rounding': 4}